# YouTube Script Sentiment Analysis

- KOSELF 기존 감성 사전 + KOSELF와 유사도 높은 단어(뉴스기사 or FastText 모델 내부)를 추가한 감성 사전

- KOSELF와 뉴스 기사 단어 유사도 정도에 따른 감성 점수 비교

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# DB 연결
import pymysql

# 실시간 주식가격 데이터
import FinanceDataReader as fdr

# 텍스트 분석
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from konlpy.tag import Okt
from gensim import models

from wordcloud import WordCloud

# 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

# 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

# * 함수 선언

## (1) set_data() - YouTube Data version 수정

In [172]:
# 출처 '자체제작 감성어휘사전 모듈_ver2.ipynb'

########## 변경 사항 ##########
## ['date_h'] 생성 하지 않음
## 날짜 기준 내림차순으로 정렬 ??
## 15시 이후 날짜 조정 없음
## print 추가 등


def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
##     df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
## 내림차순으로 변경 안 됨 => 해결 못 함
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬 
    
    if com_num != 0:
    #     통합 데이터 활용시 업종코드 지정
        df = df[df['st_cd'] == com_num] # 해당 회사만 추출
        num = str(com_num).zfill(6) # 종목코드를 6자리로 만들어줌
        
    else:
        # 특정 업종만 할 때
        num = str(df['st_cd'].iloc[0]).zfill(6)
    
    # 널값 제거
    df = df.dropna()

## 날짜 조정 불필요
#     # ============== 날짜 조정
#     ## 전일 15시 ~ 금일 15시로 날짜 조정
#     after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']
    
#     for i in range(len(df)):
#         if df['date_h'].iloc[i] in after_market:
#             df['date_d'].iloc[i] += datetime.timedelta(1)
    
    # 주말 및 공휴일 데이터
    ### Holidays
    try:
        # 서버가 열려있을 때
        db = pymysql.connect(user='root',
                             passwd='1234',
                             host='3.35.70.166',
                             db='proj',
                             charset='utf8')

        cursor = db.cursor(pymysql.cursors.DictCursor)

        sql = "select * from holidays"
        cursor.execute(sql)
        result = cursor.fetchall()
    
        # DataFrame으로 변경
        holi = pd.DataFrame(result)
        # db 닫기 --> 안하면 메모리 잡아먹음
        db.close()
    except:
        # 서버 없을 때 깃허브에서 바로 가져옴
        db_holi = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/holidays.csv'
        holi = pd.read_csv(db_holi)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for i in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][i] += datetime.timedelta(1)
     
    
    
    # 주식 데이터 가져오기
    start_date = str(df['date_d'].iloc[0])[:10].replace('-', '')
    end_date = str(df['date_d'].iloc[-1])[:10].replace('-', '')  # 뉴스 데이터 기준 최근 값까지의 주식 데이터
    
    stock = fdr.DataReader(num, start = start_date, end = end_date)
    
## change original value 그대로 사용_210924
#     # change 컬럼 판단 기준 close에서 수수료 보다 많은 비율이 올랐으면 1, 떨어졌으면 -1, 그 사이 0
#     point = stock['Close'] * stock['Change']
#     spli = stock['Close'] * 0.001  # 수수료가 0.1 % 라고 할 때
    
#     stock['ud'] = np.where( point > spli, 1,
#                           np.where(point < spli, -1, 0))

## 보합 반영 기준 고민 중_210924
    stock['ud'] = np.where( stock['Change'] > 0, 1,
                           np.where( stock['Change'] < 0, -1, 0))

    
    stock = stock.reset_index() # 날짜 컬럼 가져오기
    
    s_1 = stock[['Date','ud']] # 필요한 컬럼만 가져오기
    
    s_1.columns = ['date_d','ud'] # 컬럼 명 초기화
    
    # 뉴스데이터와 주식데이터 merge
    df_1 = pd.merge(df,s_1,on='date_d')  # 영향받는 컬럼 기준으로 inner 조인 
    
## 날짜순 정렬 (위에서 내림차순 정렬 안 됨)
#     df_1 = df_1.sort_values(by=['date_d'], axis=0, ascending=False)
    
## print 추가
    print('데이터 셋 구성 완료')
    
    return df_1

# * 데이터 로드

(1) DB에서 CSV 파일 다운로드  
(2) DB, Django 직접 접근?

## (1) DB에서 CSV 파일 다운로드  

In [7]:
# # 출처 '데이터 csv파일로 저장.ipynb'

# com_list = {'하이닉스' : '000660',
#            '현대차' : '005380',
#            '삼성전자' : '005930',
#            'LG화학' : '051910',
#            '셀트리온' : '068270'}
# media_list = {'아시아경제_' : 'asia_',
#              '매일경제_': 'maeil_',
#              'youtube_한국경제_':'hk_',
#              'youtube_삼프로_':'sampro_',
#              'youtube_슈카_':'suka_'}

# for n,m in media_list.items():
#     for i,j in com_list.items():

#         db = pymysql.connect(
#             user='root', 
#             passwd='1234', 
#             host='3.35.70.166', 
#             db='proj', 
#             charset = 'utf8'
#         )

#         cursor = db.cursor(pymysql.cursors.DictCursor)
#         if 'youtube' in n:
#             sql = "select * from youtube_"+ m + j +" where length(date) = 10"
#         else:
#             sql = "select * from "+ m +"news_craw_" + j +" where length(date) = 10"  # date 널값 제외
#         cursor.execute(sql)

#         result = cursor.fetchall()

#         # 데이터 프레임으로 변경
#         df = pd.DataFrame(result)

#         # csv 저장
#         df.to_csv('df_'+ n + i + '.csv', index=False)

In [3]:
com_list = {'하이닉스' : '000660',
           '현대차' : '005380',
           '삼성전자' : '005930',
           'LG화학' : '051910',
           '셀트리온' : '068270'}

## media_list에서 youtube list만 가져오기
youtube_list = {'youtube_한국경제_':'hk_', 
                'youtube_삼프로_':'sampro_',
                'youtube_슈카_':'suka_'}

company_list = ['하이닉스', '현대차', '삼성전자', 'LG화학', '셀트리온']

In [118]:
## csv 데이터 로드 & 데이터프레임명 - shape 출력

for n,m in youtube_list.items():
    for i,j in com_list.items():
        if i == '하이닉스':
            globals()['{}skhynix'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}skhynix'.format(m), globals()['{}skhynix'.format(m)].shape)
            
        elif i == '현대차':
            globals()['{}hyundai'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}hyundai'.format(m), globals()['{}hyundai'.format(m)].shape)
            
        elif i == '삼성전자':
            globals()['{}samsung'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}samsung'.format(m), globals()['{}samsung'.format(m)].shape)
            
        elif i == 'LG화학':
            globals()['{}lgchem'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}lgchem'.format(m), globals()['{}lgchem'.format(m)].shape)
            
        else:
            globals()['{}celltrion'.format(m)] = pd.read_csv('df_'+ n + i + '.csv')
            print('{}celltrion'.format(m), globals()['{}celltrion'.format(m)].shape)
            print()

hk_skhynix (509, 10)
hk_hyundai (530, 10)
hk_samsung (553, 10)
hk_lgchem (503, 10)
hk_celltrion (485, 10)

sampro_skhynix (494, 10)
sampro_hyundai (528, 10)
sampro_samsung (544, 10)
sampro_lgchem (508, 10)
sampro_celltrion (467, 10)

suka_skhynix (90, 10)
suka_hyundai (403, 10)
suka_samsung (448, 10)
suka_lgchem (64, 10)
suka_celltrion (197, 10)



---

# * 날짜별 데이터 분리

### 특정 날짜 데이터만 가지고 오기 ?? 
- '특정 날짜' & '모든 종목' 데이터 로드
- '특정 날짜' & '모든 유튜버' 데이터 로드

In [10]:
sampro_lgchem['date_d'].value_counts()

2021-09-07    9
2021-09-09    7
2020-09-17    7
2021-01-04    6
2021-08-24    6
             ..
2020-12-17    1
2020-12-18    1
2020-12-21    1
2020-12-22    1
2021-09-15    1
Name: date_d, Length: 252, dtype: int64

In [11]:
sampro_lgchem['date_d'].value_counts().describe()

count    252.000000
mean       2.007937
std        1.350792
min        1.000000
25%        1.000000
50%        2.000000
75%        2.250000
max        9.000000
Name: date_d, dtype: float64

In [ ]:
# 특정 월만 가져오기 ??
# str(sampro_lgchem['date_d'][0])[:7]

In [12]:
# 날짜 순 정렬 TEST => set_data 함수 
sampro_lgchem = sampro_lgchem.sort_values(by=['date_d'], axis=0, ascending=False)

In [13]:
sampro_lgchem

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
505,LG화학,51910,삼프로TV_경제의신과함께,2021091500,[증시 염탐정] 양 시장 강보합 출발.. 대형주 움직임은? f.이베스트투자증권 염승...,염 분리 염 승환 이사님 어서오십시오 4 5 4 5 4 우리 염소한 이사님과 함께 ...,25809,1850,삼프로TV제작사및멤버사공개채용세부공고확인링크httpstlySR1n쿨썸머여름특가반값세...,https://www.youtube.com/watch?v=e91KGwZ3wFc,2021-09-15,-1
504,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[퇴근길 라이브&백브리핑] 카카오 집어삼킨 금융소비자보호법, 지금 칼 빼든 이유는?...",[음악]으 으cool으[음악]으[음악][음악]투자를 하는 사람들의 깊이 있는데 와3...,188383,9580,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=0nsLNwKL40U,2021-09-14,-1
503,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[글로벌마켓 브리핑] 중국 압박하는 미국정부, 정상간 통화의 기대가 실망으로 f. ...",4 실시간 티칭 창을 봤더니 우리주님이 아침에 오라고 해서 표정 동안이 어떻게 이렇...,10914,1730,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=wJz_JpmgTO4,2021-09-14,-1
502,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 염탐정] 양 시장 강세 출발.. 들어오는 외국인, 시장 안정되나? f.이베스...",이 황구 보시고 오셨습니다 오늘도언제나처럼 우리 증지 엄 탐정 1특수 1권의 염수 ...,39754,1537,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=XoLozRQFmdo,2021-09-14,-1
501,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"우주산업의 혁신, 이제 민간이 주도한다_글로벌 라이브 _글로벌 라이브_ 장의성, 서...",[음악][음악]으으으222[음악]글로벌 라이브 에 오신걸 환영합니다안녕하십니까 박종...,86620,6295,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=8L5755txAYs,2021-09-14,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
4,LG화학,51910,삼프로TV_경제의신과함께,2020010600,[심층인터뷰] '전기차'를 중심으로 2020 자동차 시장 전망,이번 조합이 이번주 금요일이 줘 4열도에서 김한진 김일구 두니우리 짐 동안 프로 김...,9730,2036,프리미엄교육자세히알아보기클릭httpswwwpage2krnotice77신과함께가파헤친...,https://www.youtube.com/watch?v=lkvLhdCBWE4,2020-01-06,-1
3,LG화학,51910,삼프로TV_경제의신과함께,2019110800,"삼성의 스마트폰 전략, 변화의 이유는? _19.11.07_박종일,박제영 [퇴근길 P...",깨알 페이지 투4 안녕하십니까 11월 7일 퇴근 낄 페이지 툰 관세 장벽을 일부 허...,11245,4587,삼성의스마트폰전략변화의이유는191107박종일박제영퇴근길Page2,https://www.youtube.com/watch?v=fE3g9xbQ4Ro,2019-11-08,1
2,LG화학,51910,삼프로TV_경제의신과함께,2019110300,주식 포트폴리오 운용 방법! 경마 분석법(?) [최준철 대표의 주식 FM가이드 05-1],5 으주식 시장의 야전사령관 최준철 대표 함께하는 주식 sm 카이드 다섯번째시간이 ...,26347,3240,최준철대표의주식FM가이드051공격개시1부,https://www.youtube.com/watch?v=ibHnxFQxYlo,2019-11-04,1
1,LG화학,51910,삼프로TV_경제의신과함께,2019090300,"누가 마지막까지 투자하는가!_19.09.03_김효진,곽상준 [오늘아침 Page2]",4[음악]깊이가 나는 경제 컨텐츠 오늘 아침 페이지 투 정용진 김도완 입니다[음악]...,13446,4244,뉴스3첫번째뉴스자막오기로SKLG로정정합니다누가마지막까지투자하는가오늘아침Page220...,https://www.youtube.com/watch?v=Hz6pPEV4tT8,2019-09-03,1


In [14]:
# lg chemical [날짜]

lgc_210914_raw = sampro_lgchem[sampro_lgchem['date_d'] == '2021-09-14']
print(lgc_210914_raw.shape)
lgc_210914_raw.head()

(5, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
504,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[퇴근길 라이브&백브리핑] 카카오 집어삼킨 금융소비자보호법, 지금 칼 빼든 이유는?...",[음악]으 으cool으[음악]으[음악][음악]투자를 하는 사람들의 깊이 있는데 와3...,188383,9580,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=0nsLNwKL40U,2021-09-14,-1
503,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[글로벌마켓 브리핑] 중국 압박하는 미국정부, 정상간 통화의 기대가 실망으로 f. ...",4 실시간 티칭 창을 봤더니 우리주님이 아침에 오라고 해서 표정 동안이 어떻게 이렇...,10914,1730,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=wJz_JpmgTO4,2021-09-14,-1
502,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 염탐정] 양 시장 강세 출발.. 들어오는 외국인, 시장 안정되나? f.이베스...",이 황구 보시고 오셨습니다 오늘도언제나처럼 우리 증지 엄 탐정 1특수 1권의 염수 ...,39754,1537,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=XoLozRQFmdo,2021-09-14,-1
501,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"우주산업의 혁신, 이제 민간이 주도한다_글로벌 라이브 _글로벌 라이브_ 장의성, 서...",[음악][음악]으으으222[음악]글로벌 라이브 에 오신걸 환영합니다안녕하십니까 박종...,86620,6295,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=8L5755txAYs,2021-09-14,-1
500,LG화학,51910,삼프로TV_경제의신과함께,2021091400,"[증시 셔터맨] 쏠림이 심한 국내 주식, 중심 잃지 않는 종목 선정이 필요하다. ...",4아침에 도착한 말씀드렸습니다만u 머리가 이상해 보인다면여러분의 패션 감각을 의심해...,21914,1573,삼프로TV제작사및멤버사공개채용세부공고확인링크httpswwwpage2krnotice1...,https://www.youtube.com/watch?v=JOK1oGTKi1U,2021-09-14,-1


In [ ]:
suka_cell = suka_celltrion.head()

In [ ]:
# suka_cell_sd = set_data(suka_cell)

# * (2) text_pre() & stop_words() 

## youtube_pre 로 합쳐서 만들기(예정)
- 소요 시간 표시 추가(예정)

In [144]:
# 출처 '자체제작 감성어휘사전 모듈_ver2.ipynb' > def giro_dic
##표시 수정 부분

def text_pre(df,num = 0):

#     df = set_data(df, num)
#     print('데이터 셋 구성 완료')
    
    okt = Okt()
    print('okt 모듈 활성')
    
    
    # 데이터 전처리
    # =================== nlp
    # 1. 제목, 텍스트 --> 명사, 형용사 추출
    # 태그 삭제        
    # \n \t \s 삭제
## print문 추가1
    print('태그 삭제 중')
    df['text'] = df['text'].str.replace('[\n|\t|\r|\s]','')
    df['text'] = df['text'].str.replace('[^a-zA-Z0-9가-힣]','')

## print문 추가2
    print('품사 태깅 작업 중')    
    # 토큰화 및 품사 태깅
    df['Contents_input'] = df['text'].apply(lambda x: okt.pos(x, norm=True, stem=True))
    print('태그 삭제 및 품사 태깅 완료')
    
    # 명사, 형용사 추출
    for i in range(len(df['Contents_input'])):
        wordList = []
        for j in range(len(df['Contents_input'][i])):
            if df['Contents_input'][i][j][1] == 'Noun' or df['Contents_input'][i][j][1] == 'Adjective':
                wordList.append(df['Contents_input'][i][j][0])         
        df['Contents_input'][i] = wordList
    print('명사, 형용사 추출')
    
## print문 추가3
    print('text_pre 작업 완료!')
    
    return df

In [ ]:
''' # 추후 미디어별 업데이트
media = ['오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
         '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
         '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
         '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']

## 함수 내 media 전역 변수 선언
#     global media
'''

In [6]:
def stop_words(df,num = 0):
    
## media 전역 변수 선언
#     global media
    
    # 2. 불용어 처리
    # 불용어 사전 로딩
##     url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/stopwords_ver1.txt'
    url = 'https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt'
    stopwords = list(pd.read_csv(url, header=None)[0])
##     media = ['매일경제','mkcokr','무단전재및재배포금지','무단전재','재배포금지']
    media = ['실시간', '오늘', '년도', '이제', '요즘', '지금', '우리나라', '한번', '사실', '처럼', \
             '증시', '개장', '얘기', '이렇다', '참고', '없다', '때문', '아니다', \
             '안녕하십니까', '안녕하세요우', '안녕하다', '어서', '오십', '시오', '모습', '내용', '정도', \
             '그렇다', '어쨌든', '어떻다', '박지영', '미래에셋', '어머님', '음악']
    print('불용어 사전 로딩')
    
    
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in stopwords:
                df['Contents_input'][i].remove(word)
    print('불용어 1차 처리 완료')
    
## for문 분리                
    for i in range(len(df['Contents_input'])):
        for word in df['Contents_input'][i]:
            if word in media:
                df['Contents_input'][i].remove(word) 
    print('불용어 2차 처리 완료')
    

## 단어 길이 1 이하 제거

    
    # (1) 단어길이가 1이하인 단어 제거
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        for word in wordList:
            if len(word) <= 1:
                df['Contents_input'][index].remove(word)
    print('단어 길이 1이하인 단어 제거')
    

    ''' TEST 결과 안 됨
    # (2) 단어 길이가 1 초과인 단어
    for index, wordList in enumerate(df['Contents_input']):
        if len(df['Contents_input'][index]) == 0:
            continue
        wordList = [x for x in wordList if len(x) > 1]
    print('단어 길이 2이상인 단어 리스트 생성')    
    '''

## print문 추가4
    print('stop_words 작업 완료!')
    
    return df

# 목표 : STT 텍스트 정제

## FastText

https://inahjeon.github.io/fasttext/

In [29]:
from gensim.models import FastText

In [ ]:
# gunzip cc.ko.300.vec.gz
# gunzip cc.ko.300.bin.gz

In [30]:
ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [ ]:
for w, sim in m_fasttext.similar_by_word('파이썬', 10):
    print(f'{w}: {sim}')

In [ ]:
model = FastText(data, size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
model.wv.most_similar('')

https://vhrehfdl.tistory.com/77

In [32]:
# embedding = FastText(lgc_210914['Contents_input'][0], size=100, window=7, negative=3, min_count=5)
embedding = FastText(lgc_210914['Contents_input'][0], window=7, negative=3, min_count=5)
embedding.save('fasttext.model')

In [33]:
model = FastText.load('fasttext.model')

In [34]:
# print(model.wv['티칭창'])

[-2.5865317e-03 -3.8800368e-04  3.5524275e-03 -2.1607096e-03
 -2.3786856e-03  7.1477302e-04  1.7091107e-03  1.2328442e-03
 -2.7699741e-03  2.4295569e-04 -1.7320713e-03  2.0132323e-03
  3.4837208e-03 -7.4854051e-04 -2.6100241e-03  8.2819071e-04
 -6.3785445e-04  5.2880780e-03 -1.3630780e-03  2.4474559e-03
  2.0023251e-03 -3.0574251e-03  3.5545893e-03  1.6764858e-03
  2.2446816e-03  2.6308221e-03 -3.2320537e-04 -3.8382588e-03
  3.2423770e-03 -1.9063704e-03 -3.8454544e-03  1.0989885e-03
 -2.7893891e-03 -1.6144030e-03  5.0837075e-04  1.1414988e-03
  2.7172960e-04 -3.5560690e-03 -1.5779581e-03 -8.5963117e-04
 -1.6861191e-03 -1.6496499e-03 -1.8934374e-03 -3.4711186e-03
  2.1728599e-03 -3.0230763e-04 -1.9615588e-03 -1.6256232e-03
  2.5016570e-03  1.1014943e-03 -1.4571085e-03 -9.2227594e-04
  1.2993051e-04 -2.8032620e-04  1.3599679e-03  2.4908383e-03
 -5.8836554e-04 -7.1876422e-03  3.4908373e-03  3.9292860e-04
 -1.4511369e-03  4.3322914e-03 -2.3795899e-03 -1.3235770e-04
 -1.0918585e-03 -4.70315

In [36]:
print(model.wv.most_similar('티칭창'))

[('힘', 0.2857452929019928), ('레', 0.268752783536911), ('음', 0.25929611921310425), ('혁', 0.24410496652126312), ('계', 0.2408999502658844), ('츠', 0.23554031550884247), ('순', 0.23359213769435883), ('인', 0.2310313731431961), ('향', 0.22450362145900726), ('응', 0.2218310832977295)]


## Py-Hansell

# 감성 점수 계산

### 감성 사전 로드

In [ ]:
#     # 주식 감성 사전 로딩 : koself
#     koself_pro = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/KOSELF_pos.txt'
#     koself_neg = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/data/KOSELF_neg.txt'
    
#     positive = list(pd.read_csv(koself_pro, header=None)[0])
#     negative = list(pd.read_csv(koself_neg, header=None)[0])
#     print('koself 사전 로딩...')
    
#     # koself 만 쓴 감성점수
#     df['koself'] = 0
#     for index, wordList in enumerate(df['Contents_input']):
#         for word in wordList:
#             if word in positive:
#                 df['koself'][index] += 1
#             if word in negative:
#                 df['koself'][index] -= 1
#     print('koself 점수 계산 완료')


In [171]:
com_list = {'하이닉스' : '000660',
           '현대차' : '005380',
           '삼성전자' : '005930',
           'LG화학' : '051910',
           '셀트리온' : '068270'}

# KOSELF 사전 어휘와의 유사도
similar = ['50', '60', '65', '70']


for i, j in com_list.items():

    if i == '하이닉스':
        com = 'sk'
        print('###############################################################   <<  SK Hynix Lexicon  >>  ###############################################################')

        for sent in ['pos', 'neg']:
            for sim in similar:
                with open('Lexicon/lexicon_' + com + '_' + sent + '_' + sim + '.txt') as f:
                    lexicon = f.readlines()
                globals()['{0}_{1}_{2}'.format(com, sent, sim)] = [word.replace('\n', '') for word in lexicon]
                print('\n *', com, sent, '유사도', sim, '사전 어휘 수 : ', len(globals()['{0}_{1}_{2}'.format(com, sent, sim)]), '개', '\n\n', globals()['{0}_{1}_{2}'.format(com, sent, sim)])
        print('\n##########################################################################################################################################################\n\n')

        
    elif i == '현대차':
        com = 'hyundai'
        print('###############################################################   <<  Hyundai Lexicon  >>  ###############################################################')

        for sent in ['pos', 'neg']:
            for sim in similar:
                with open('Lexicon/lexicon_' + com + '_' + sent + '_' + sim + '.txt') as f:
                    lexicon = f.readlines()
                globals()['{0}_{1}_{2}'.format(com, sent, sim)] = [word.replace('\n', '') for word in lexicon]
                print('\n *', com, sent, '유사도', sim, '사전 어휘 수 : ', len(globals()['{0}_{1}_{2}'.format(com, sent, sim)]), '개', '\n\n', globals()['{0}_{1}_{2}'.format(com, sent, sim)])
        print('\n##########################################################################################################################################################\n\n')


    elif i == '삼성전자':
        com = 'samsung'
        print('###############################################################   <<  Samsung Lexicon  >>  ###############################################################')

        for sent in ['pos', 'neg']:
            for sim in similar:
                with open('Lexicon/lexicon_' + com + '_' + sent + '_' + sim + '.txt') as f:
                    lexicon = f.readlines()
                globals()['{0}_{1}_{2}'.format(com, sent, sim)] = [word.replace('\n', '') for word in lexicon]
                print('\n *', com, sent, '유사도', sim, '사전 어휘 수 : ', len(globals()['{0}_{1}_{2}'.format(com, sent, sim)]), '개', '\n\n', globals()['{0}_{1}_{2}'.format(com, sent, sim)])
        print('\n##########################################################################################################################################################\n\n')


    elif i == 'LG화학':
        com = 'lg'
        print('###############################################################   <<  LG Chem Lexicon  >>  ###############################################################')

        for sent in ['pos', 'neg']:
            for sim in similar:
                with open('Lexicon/lexicon_' + com + '_' + sent + '_' + sim + '.txt') as f:
                    lexicon = f.readlines()
                globals()['{0}_{1}_{2}'.format(com, sent, sim)] = [word.replace('\n', '') for word in lexicon]
                print('\n *', com, sent, '유사도', sim, '사전 어휘 수 : ', len(globals()['{0}_{1}_{2}'.format(com, sent, sim)]), '개', '\n\n', globals()['{0}_{1}_{2}'.format(com, sent, sim)])
        print('\n##########################################################################################################################################################\n\n')

        
    elif i == '셀트리온':
        com = 'celltrion'
        print('##############################################################   <<  Celltrion Lexicon  >>  ##############################################################')

        for sent in ['pos', 'neg']:
            for sim in similar:
                with open('Lexicon/lexicon_' + com + '_' + sent + '_' + sim + '.txt') as f:
                    lexicon = f.readlines()
                globals()['{0}_{1}_{2}'.format(com, sent, sim)] = [word.replace('\n', '') for word in lexicon]
                print('\n *', com, sent, '유사도', sim, '사전 어휘 수 : ', len(globals()['{0}_{1}_{2}'.format(com, sent, sim)]), '개', '\n\n', globals()['{0}_{1}_{2}'.format(com, sent, sim)])
        print('\n##########################################################################################################################################################\n\n')


###############################################################   <<  SK Hynix Lexicon  >>  ###############################################################

 * sk pos 유사도 50 사전 어휘 수 :  416 개 

 ['가치', '가치 있는', '강세', '개선', '개선된', '개선되는', '경신', '경의', '고급', '기꺼이', '더 좋은', '도움이 되는', '뛰어나', '명성', '바닥', '본격적인', '상승효과', '상회한', '성공', '수혜', '순조롭게', '실현', '안정된', '완전한', '우세', '우월', '우호적', '의심의 여지가 없는', '이끄는', '이로운', '인상적인', '장점', '적극', '적극적으로', '정성', '중요한', '증대', '최고점', '최선', '추진', '추천', '충분히', '충족', '해결', '호황', '확보', '활력', '활성화', '문무역', '프로그램매매', '정용배', '특징', '코팅덕트', '엄재철', '밑돈', '증진', '밸류맵', '긴급물량', '닛케이지수', '이주수', '훈풍', '허페이시', '수펙스홀', '배팅했다', '윤제용', '이의섭', '팔리흐', '불경기', '소재효', '게임스탑', '박학주', '송출되', '첵랍콕국제공항', '외부펀딩', '씽크빅웅진씽크빅', '코스포켐텍', '티라유텍티라유텍', '토픽스지수', '빅프라핏', '스냅드래건', '격이므', '폐쇄되', '견조했', '비밀보호법', '상괄르', '생뚱맞', '충북지방조달청', '횡보등락할', '맥쿼리인프라', '김혜일', '발전', '안중근함', '신해용', '랙티브', '배정물량', '사대부중', '엠앤티삼강엠', '류지민', '박도수', '엄주용', '증대', '퍼부으', '가장', '남완호', '엠비덱스', '스제약위더스', '샘코샘코', '나노메딕스', '김윤욱

In [ ]:
# hk_skhynix (509, 10)
# hk_hyundai (530, 10)
# hk_samsung (553, 10)
# hk_lgchem (503, 10)
# hk_celltrion (485, 10)

# sampro_skhynix (494, 10)
# sampro_hyundai (528, 10)
# sampro_samsung (544, 10)
# sampro_lgchem (508, 10)
# sampro_celltrion (467, 10)

# suka_skhynix (90, 10)
# suka_hyundai (403, 10)
# suka_samsung (448, 10)
# suka_lgchem (64, 10)
# suka_celltrion (197, 10)

In [132]:
sampro_skhynix = set_data(sampro_skhynix)
sampro_hyundai = set_data(sampro_hyundai)
sampro_samsung = set_data(sampro_samsung)
sampro_lgchem = set_data(sampro_lgchem)
sampro_celltrion = set_data(sampro_celltrion)

데이터 셋 구성 완료
데이터 셋 구성 완료
데이터 셋 구성 완료
데이터 셋 구성 완료
데이터 셋 구성 완료


In [179]:
sampro_210924 = sampro_skhynix[sampro_skhynix['date_d'] == '2021-09-24']
print(sampro_210924.ud)
sampro_210924 = sampro_hyundai[sampro_hyundai['date_d'] == '2021-09-24']
print(sampro_210924.ud)
sampro_210924 = sampro_samsung[sampro_samsung['date_d'] == '2021-09-24']
print(sampro_210924.ud)
sampro_210924 = sampro_lgchem[sampro_lgchem['date_d'] == '2021-09-24']
print(sampro_210924.ud)
sampro_210924 = sampro_celltrion[sampro_celltrion['date_d'] == '2021-09-24']
print(sampro_210924.ud)

492   -1
Name: ud, dtype: int32
Series([], Name: ud, dtype: int32)
542   -1
543   -1
Name: ud, dtype: int32
507    1
Name: ud, dtype: int32
465   -1
466   -1
Name: ud, dtype: int32


## (1) 삼프로_하이닉스 감성 점수

In [82]:
# Lexicon 로드
print('#######################################################################################################')
print('####                                   Lexicon SK Hynix Postive                                   #####')
print('#######################################################################################################')


with open('Lexicon/lexicon_sk_pos_50.txt') as f:
    lexicon = f.readlines()
sk_pos_50 = [word.replace('\n', '') for word in lexicon]
print('\n* 유사도 0.5 사전 어휘 수 : ', len(sk_pos_50), '개', '\n\n', sk_pos_50)

with open('Lexicon/lexicon_sk_pos_60.txt') as f:
    lexicon = f.readlines()
sk_pos_60 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.6 사전 어휘 수 : ', len(sk_pos_60), '개', '\n\n', sk_pos_60)

with open('Lexicon/lexicon_sk_pos_65.txt') as f:
    lexicon = f.readlines()
sk_pos_65 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.65 사전 어휘 수 : ', len(sk_pos_65), '개', '\n\n', sk_pos_65)

with open('Lexicon/lexicon_sk_pos_70.txt') as f:
    lexicon = f.readlines()
sk_pos_70 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.7 사전 어휘 수 : ', len(sk_pos_70), '개', '\n\n', sk_pos_70)


print('\n\n')
print('#######################################################################################################')
print('####                                   Lexicon SK Hynix Negative                                  #####')
print('#######################################################################################################')


with open('Lexicon/lexicon_sk_neg_50.txt') as f:
    lexicon = f.readlines()
sk_neg_50 = [word.replace('\n', '') for word in lexicon]
print('\n* 유사도 0.5 사전 어휘 수 : ', len(sk_neg_50), '개', '\n\n', sk_neg_50)

with open('Lexicon/lexicon_sk_neg_60.txt') as f:
    lexicon = f.readlines()
sk_neg_60 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.6 사전 어휘 수 : ', len(sk_neg_60), '개', '\n\n', sk_neg_60)

with open('Lexicon/lexicon_sk_neg_65.txt') as f:
    lexicon = f.readlines()
sk_neg_65 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.65 사전 어휘 수 : ', len(sk_neg_65), '개', '\n\n', sk_neg_65)

with open('Lexicon/lexicon_sk_neg_70.txt') as f:
    lexicon = f.readlines()
sk_neg_70 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.7 사전 어휘 수 : ', len(sk_neg_70), '개', '\n\n', sk_neg_70)

#######################################################################################################
####                                   Lexicon SK Hynix Postive                                   #####
#######################################################################################################

* 유사도 0.5 사전 어휘 수 :  416 개 

 ['가치', '가치 있는', '강세', '개선', '개선된', '개선되는', '경신', '경의', '고급', '기꺼이', '더 좋은', '도움이 되는', '뛰어나', '명성', '바닥', '본격적인', '상승효과', '상회한', '성공', '수혜', '순조롭게', '실현', '안정된', '완전한', '우세', '우월', '우호적', '의심의 여지가 없는', '이끄는', '이로운', '인상적인', '장점', '적극', '적극적으로', '정성', '중요한', '증대', '최고점', '최선', '추진', '추천', '충분히', '충족', '해결', '호황', '확보', '활력', '활성화', '문무역', '프로그램매매', '정용배', '특징', '코팅덕트', '엄재철', '밑돈', '증진', '밸류맵', '긴급물량', '닛케이지수', '이주수', '훈풍', '허페이시', '수펙스홀', '배팅했다', '윤제용', '이의섭', '팔리흐', '불경기', '소재효', '게임스탑', '박학주', '송출되', '첵랍콕국제공항', '외부펀딩', '씽크빅웅진씽크빅', '코스포켐텍', '티라유텍티라유텍', '토픽스지수', '빅프라핏', '스냅드래건', '격이므', '폐쇄되', '견조했', '비밀보호법', '상괄르', '생뚱맞', '충북지방조달청', '횡보등락할', '맥쿼리인프라'

In [101]:
sampro_skhynix = set_data(sampro_skhynix)

데이터 셋 구성 완료


In [102]:
sampro_210924 = sampro_skhynix[sampro_skhynix['date_d'] == '2021-09-24']
print(sampro_210924.shape)
sampro_210924.head()

(1, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
492,하이닉스,660,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위 헌터 증권의 박진희 우정과오늘마감 이왕 정리를 좀 해 드리죠토스 쭉정이 안녕하...,5949,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1


In [103]:
%%time

sampro_210924 = text_pre(sampro_210924)
sampro_210924 = stop_words(sampro_210924)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 중
품사 태깅 작업 중
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
text_pre 작업 완료!
불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 1이하인 단어 제거
stop_words 작업 완료!
Wall time: 17.5 s


In [104]:
sampro_210924 

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input
0,하이닉스,660,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5949,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,..."


In [ ]:
# sk_pos_50
# sk_pos_60
# sk_pos_65
# sk_pos_70

# sk_neg_50
# sk_neg_60
# sk_neg_65
# sk_neg_70

In [105]:
pos = sk_pos_50
neg = sk_neg_50

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,하이닉스,660,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5949,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",14


In [106]:
pos = sk_pos_60
neg = sk_neg_60

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,하이닉스,660,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5949,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-2


In [107]:
pos = sk_pos_65
neg = sk_neg_65

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,하이닉스,660,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5949,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-6


In [108]:
pos = sk_pos_70
neg = sk_neg_70

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,하이닉스,660,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5949,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-2


## (2) 삼프로_삼성전자 감성 점수

In [89]:
print('#######################################################################################################')
print('####                                   Lexicon Samsung Postive                                    #####')
print('#######################################################################################################')


with open('Lexicon/lexicon_samsung_pos_50.txt') as f:
    lexicon = f.readlines()
ss_pos_50 = [word.replace('\n', '') for word in lexicon]
print('\n* 유사도 0.5 사전 어휘 수 : ', len(ss_pos_50), '개', '\n\n', ss_pos_50)

with open('Lexicon/lexicon_samsung_pos_60.txt') as f:
    lexicon = f.readlines()
ss_pos_60 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.6 사전 어휘 수 : ', len(ss_pos_60), '개', '\n\n', ss_pos_60)

with open('Lexicon/lexicon_samsung_pos_65.txt') as f:
    lexicon = f.readlines()
ss_pos_65 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.65 사전 어휘 수 : ', len(ss_pos_65), '개', '\n\n', ss_pos_65)

with open('Lexicon/lexicon_samsung_pos_70.txt') as f:
    lexicon = f.readlines()
ss_pos_70 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.7 사전 어휘 수 : ', len(ss_pos_70), '개', '\n\n', ss_pos_70)


print('\n\n')
print('#######################################################################################################')
print('####                                   Lexicon Samsung Negative                                   #####')
print('#######################################################################################################')


with open('Lexicon/lexicon_samsung_neg_50.txt') as f:
    lexicon = f.readlines()
ss_neg_50 = [word.replace('\n', '') for word in lexicon]
print('\n* 유사도 0.5 사전 어휘 수 : ', len(ss_neg_50), '개', '\n\n', ss_neg_50)

with open('Lexicon/lexicon_samsung_neg_60.txt') as f:
    lexicon = f.readlines()
ss_neg_60 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.6 사전 어휘 수 : ', len(ss_neg_60), '개', '\n\n', ss_neg_60)

with open('Lexicon/lexicon_samsung_neg_65.txt') as f:
    lexicon = f.readlines()
ss_neg_65 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.65 사전 어휘 수 : ', len(ss_neg_65), '개', '\n\n', ss_neg_65)

with open('Lexicon/lexicon_samsung_neg_70.txt') as f:
    lexicon = f.readlines()
ss_neg_70 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.7 사전 어휘 수 : ', len(ss_neg_70), '개', '\n\n', ss_neg_70)

#######################################################################################################
####                                   Lexicon Samsung Postive                                    #####
#######################################################################################################

* 유사도 0.5 사전 어휘 수 :  1160 개 

 ['가치', '가치 있는', '강세', '개선', '개선된', '개선되는', '경신', '경의', '고급', '기꺼이', '더 좋은', '도움이 되는', '뛰어나', '명성', '바닥', '본격적인', '상승효과', '상회한', '성공', '수혜', '순조롭게', '실현', '안정된', '완전한', '우세', '우월', '우호적', '의심의 여지가 없는', '이끄는', '이로운', '인상적인', '장점', '적극', '적극적으로', '정성', '중요한', '증대', '최고점', '최선', '추진', '추천', '충분히', '충족', '해결', '호황', '확보', '활력', '활성화', '런정페이', '별정통신사', '딜라이트숍', '개월안', '중요', '메디앙스', '본격', '송영호', '부약정', '자공시', '필로헬스', '샌드블래스트', '이노뎁', '아펨스테크놀러지', '쿠보텍', '인더케그', '베른스테', '코누코피아', '더블폴딩', '공영은', '창저우시', '유데나필', '화생방방호사령부', '컨설트', '설진훈', '쿼스유비쿼스', '무르익으', '디맨드풀', '센티먼트', '인멸했다', '커넥티드버스', '리밸런싱했다', '양규석', '김민현', '아트사커', '정종필', '페이턴트리절트', '민혜련', '첵랍콕국제공항', '펠리스

In [94]:
sampro_samsung = set_data(sampro_samsung)

데이터 셋 구성 완료


In [99]:
sampro_210924 = sampro_samsung[sampro_samsung['date_d'] == '2021-09-24']
print(sampro_210924.shape)
sampro_210924.head()

(2, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
542,삼성전자,5930,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위 헌터 증권의 박진희 우정과오늘마감 이왕 정리를 좀 해 드리죠토스 쭉정이 안녕하...,5974,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1
543,삼성전자,5930,삼프로TV_경제의신과함께,2021092400,"[퇴근길 라이브 / 백브리핑 라이브] 中 헝다 사태, 美 금리 리스크, 꿀잼 大토...",[음악]으our[음악]free[음악]2[음악]자 3부 대비 시청자 여러분 그리고경제...,156788,9902,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=0zYWEIdMF24,2021-09-24,-1


In [100]:
%%time

sampro_210924 = text_pre(sampro_210924)
sampro_210924 = stop_words(sampro_210924)

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 중
품사 태깅 작업 중
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
text_pre 작업 완료!
불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 1이하인 단어 제거
stop_words 작업 완료!
Wall time: 3min 6s


In [90]:
pos = ss_pos_50
neg = ss_neg_50

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",0


In [91]:
pos = ss_pos_60
neg = ss_neg_60

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-2


In [92]:
pos = ss_pos_65
neg = ss_neg_65

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-6


In [93]:
pos = ss_pos_70
neg = ss_neg_70

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-2


## (3) 삼프로_엘지화학 감성 점수

In [119]:
sampro_lgchem = set_data(sampro_lgchem)
sampro_lgchem.head()

데이터 셋 구성 완료


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
0,LG화학,51910,삼프로TV_경제의신과함께,2019090100,주식 투자자가 버려야 할 7가지! [최준철 대표의 주식 FM가이드 #3],[음악]최진철 테스의 주식 fm 가이드세번째 시간이 시작됐습니다 반갑습니다 점프로 ...,38324,4364,주식투자자가버려야할7가지최준철대표의주식FM가이드3,https://www.youtube.com/watch?v=EsrvOHJSuTo,2019-09-02,-1
1,LG화학,51910,삼프로TV_경제의신과함께,2019090300,"누가 마지막까지 투자하는가!_19.09.03_김효진,곽상준 [오늘아침 Page2]",4[음악]깊이가 나는 경제 컨텐츠 오늘 아침 페이지 투 정용진 김도완 입니다[음악]...,13446,4244,뉴스3첫번째뉴스자막오기로SKLG로정정합니다누가마지막까지투자하는가오늘아침Page220...,https://www.youtube.com/watch?v=Hz6pPEV4tT8,2019-09-03,1
2,LG화학,51910,삼프로TV_경제의신과함께,2019110300,주식 포트폴리오 운용 방법! 경마 분석법(?) [최준철 대표의 주식 FM가이드 05-1],5 으주식 시장의 야전사령관 최준철 대표 함께하는 주식 sm 카이드 다섯번째시간이 ...,26347,3240,최준철대표의주식FM가이드051공격개시1부,https://www.youtube.com/watch?v=ibHnxFQxYlo,2019-11-04,1
3,LG화학,51910,삼프로TV_경제의신과함께,2019110800,"삼성의 스마트폰 전략, 변화의 이유는? _19.11.07_박종일,박제영 [퇴근길 P...",깨알 페이지 투4 안녕하십니까 11월 7일 퇴근 낄 페이지 툰 관세 장벽을 일부 허...,11245,4587,삼성의스마트폰전략변화의이유는191107박종일박제영퇴근길Page2,https://www.youtube.com/watch?v=fE3g9xbQ4Ro,2019-11-08,1
4,LG화학,51910,삼프로TV_경제의신과함께,2020010600,[심층인터뷰] '전기차'를 중심으로 2020 자동차 시장 전망,이번 조합이 이번주 금요일이 줘 4열도에서 김한진 김일구 두니우리 짐 동안 프로 김...,9730,2036,프리미엄교육자세히알아보기클릭httpswwwpage2krnotice77신과함께가파헤친...,https://www.youtube.com/watch?v=lkvLhdCBWE4,2020-01-06,-1


In [120]:
sampro_210924 = sampro_lgchem[sampro_lgchem['date_d'] == '2021-09-24']
print(sampro_210924.shape)
sampro_210924.head()

(1, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
507,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위 헌터 증권의 박진희 우정과오늘마감 이왕 정리를 좀 해 드리죠토스 쭉정이 안녕하...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1


In [121]:
%%time 
sampro_210924 = text_pre(sampro_210924)
sampro_210924

## ud_x, ud_y 칼럼이 생김!?

데이터 셋 구성 완료
okt 모듈 활성
태그 삭제 중
품사 태깅 작업 중
태그 삭제 및 품사 태깅 완료
명사, 형용사 추출
text_pre 작업 완료!
Wall time: 11.7 s


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 오늘, 마감, 정리, 좀, 토스, 쭉정이, ..."


In [122]:
%%time
sampro_210924 = stop_words(sampro_210924)
sampro_210924.head()

Wall time: 0 ns
불용어 사전 로딩
불용어 1차 처리 완료
불용어 2차 처리 완료
단어 길이 1이하인 단어 제거
stop_words 작업 완료!


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,..."


In [114]:
print('#######################################################################################################')
print('####                                   Lexicon LG Chem Postive                                    #####')
print('#######################################################################################################')


with open('Lexicon/lexicon_lg_pos_50.txt') as f:
    lexicon = f.readlines()
lg_pos_50 = [word.replace('\n', '') for word in lexicon]
print('\n* 유사도 0.5 사전 어휘 수 : ', len(lg_pos_50), '개', '\n\n', lg_pos_50)

with open('Lexicon/lexicon_lg_pos_60.txt') as f:
    lexicon = f.readlines()
lg_pos_60 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.6 사전 어휘 수 : ', len(lg_pos_60), '개', '\n\n', lg_pos_60)

with open('Lexicon/lexicon_lg_pos_65.txt') as f:
    lexicon = f.readlines()
lg_pos_65 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.65 사전 어휘 수 : ', len(lg_pos_65), '개', '\n\n', lg_pos_65)

with open('Lexicon/lexicon_lg_pos_70.txt') as f:
    lexicon = f.readlines()
lg_pos_70 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.7 사전 어휘 수 : ', len(lg_pos_70), '개', '\n\n', lg_pos_70)


print('\n\n')
print('#######################################################################################################')
print('####                                   Lexicon LG Chem Negative                                   #####')
print('#######################################################################################################')


with open('Lexicon/lexicon_lg_neg_50.txt') as f:
    lexicon = f.readlines()
lg_neg_50 = [word.replace('\n', '') for word in lexicon]
print('\n* 유사도 0.5 사전 어휘 수 : ', len(lg_neg_50), '개', '\n\n', lg_neg_50)

with open('Lexicon/lexicon_lg_neg_60.txt') as f:
    lexicon = f.readlines()
lg_neg_60 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.6 사전 어휘 수 : ', len(lg_neg_60), '개', '\n\n', lg_neg_60)

with open('Lexicon/lexicon_lg_neg_65.txt') as f:
    lexicon = f.readlines()
lg_neg_65 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.65 사전 어휘 수 : ', len(lg_neg_65), '개', '\n\n', lg_neg_65)

with open('Lexicon/lexicon_lg_neg_70.txt') as f:
    lexicon = f.readlines()
lg_neg_70 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.7 사전 어휘 수 : ', len(lg_neg_70), '개', '\n\n', lg_neg_70)

#######################################################################################################
####                                   Lexicon LG Chem Postive                                    #####
#######################################################################################################

* 유사도 0.5 사전 어휘 수 :  410 개 

 ['가치', '가치 있는', '강세', '개선', '개선된', '개선되는', '경신', '경의', '고급', '기꺼이', '더 좋은', '도움이 되는', '뛰어나', '명성', '바닥', '본격적인', '상승효과', '상회한', '성공', '수혜', '순조롭게', '실현', '안정된', '완전한', '우세', '우월', '우호적', '의심의 여지가 없는', '이끄는', '이로운', '인상적인', '장점', '적극', '적극적으로', '정성', '중요한', '증대', '최고점', '최선', '추진', '추천', '충분히', '충족', '해결', '호황', '확보', '활력', '활성화', '팩토바', '증진', '엔드플레이트', '윤여삼', '사대부중', '기록', '증가', '엠비엔', '윤자민', '리레이팅', '사업', '케이에스', '임지아', '박충현', '테믹시스', '하태진', '토픽스지수', '코슈메슈티컬', '레브친', '기가와트', '진행', '골든써밋', '빅테크주', '자공시', '과잉반응', '저하', '확보', '란샤오민', '여준호', '메디콕스메디콕스', '배규웅', '재생기술', '창동차량기지', '성민석', '코세스', '가치나', '오해진', '요구', '허페이시', '티라유텍티라유텍', '거버넌스위', '럭키석유화학', '방

In [123]:
pos = lg_pos_50
neg = lg_neg_50

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",0


In [124]:
pos = lg_pos_60
neg = lg_neg_60

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-2


In [125]:
pos = lg_pos_65
neg = lg_neg_65

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-6


In [126]:
pos = lg_pos_70
neg = lg_neg_70

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-2


## (4) 삼프로_셀트리온 감성 점수

In [111]:
sampro_celltrion = set_data(sampro_celltrion)

데이터 셋 구성 완료


In [112]:
sampro_210924 = sampro_celltrion[sampro_celltrion['date_d'] == '2021-09-24']
print(sampro_210924.shape)
sampro_210924.head()

(2, 12)


,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud
465,셀트리온,68270,삼프로TV_경제의신과함께,2021092400,"[퇴근길 라이브 / 백브리핑 라이브] 中 헝다 사태, 美 금리 리스크, 꿀잼 大토...",[음악]으our[음악]free[음악]2[음악]자 3부 대비 시청자 여러분 그리고경제...,155466,9902,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=0zYWEIdMF24,2021-09-24,-1
466,셀트리온,68270,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위 헌터 증권의 박진희 우정과오늘마감 이왕 정리를 좀 해 드리죠토스 쭉정이 안녕하...,5867,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1


In [127]:
print('#######################################################################################################')
print('####                                  Lexicon Celltrion Postive                                   #####')
print('#######################################################################################################')


with open('Lexicon/lexicon_celltrion_pos_50.txt') as f:
    lexicon = f.readlines()
cell_pos_50 = [word.replace('\n', '') for word in lexicon]
print('\n* 유사도 0.5 사전 어휘 수 : ', len(cell_pos_50), '개', '\n\n', cell_pos_50)

with open('Lexicon/lexicon_celltrion_pos_60.txt') as f:
    lexicon = f.readlines()
cell_pos_60 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.6 사전 어휘 수 : ', len(cell_pos_60), '개', '\n\n', cell_pos_60)

with open('Lexicon/lexicon_celltrion_pos_65.txt') as f:
    lexicon = f.readlines()
cell_pos_65 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.65 사전 어휘 수 : ', len(cell_pos_65), '개', '\n\n', cell_pos_65)

with open('Lexicon/lexicon_celltrion_pos_70.txt') as f:
    lexicon = f.readlines()
cell_pos_70 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.7 사전 어휘 수 : ', len(cell_pos_70), '개', '\n\n', cell_pos_70)


print('\n\n')
print('#######################################################################################################')
print('####                                  Lexicon Celltrion Negative                                  #####')
print('#######################################################################################################')


with open('Lexicon/lexicon_celltrion_neg_50.txt') as f:
    lexicon = f.readlines()
cell_neg_50 = [word.replace('\n', '') for word in lexicon]
print('\n* 유사도 0.5 사전 어휘 수 : ', len(cell_neg_50), '개', '\n\n', cell_neg_50)

with open('Lexicon/lexicon_celltrion_neg_60.txt') as f:
    lexicon = f.readlines()
cell_neg_60 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.6 사전 어휘 수 : ', len(cell_neg_60), '개', '\n\n', cell_neg_60)

with open('Lexicon/lexicon_celltrion_neg_65.txt') as f:
    lexicon = f.readlines()
cell_neg_65 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.65 사전 어휘 수 : ', len(cell_neg_65), '개', '\n\n', cell_neg_65)

with open('Lexicon/lexicon_celltrion_neg_70.txt') as f:
    lexicon = f.readlines()
cell_neg_70 = [word.replace('\n', '') for word in lexicon]
print('\n\n* 유사도 0.7 사전 어휘 수 : ', len(cell_neg_70), '개', '\n\n', cell_neg_70)

#######################################################################################################
####                                  Lexicon Celltrion Postive                                   #####
#######################################################################################################

* 유사도 0.5 사전 어휘 수 :  371 개 

 ['가치', '가치 있는', '강세', '개선', '개선된', '개선되는', '경신', '경의', '고급', '기꺼이', '더 좋은', '도움이 되는', '뛰어나', '명성', '바닥', '본격적인', '상승효과', '상회한', '성공', '수혜', '순조롭게', '실현', '안정된', '완전한', '우세', '우월', '우호적', '의심의 여지가 없는', '이끄는', '이로운', '인상적인', '장점', '적극', '적극적으로', '정성', '중요한', '증대', '최고점', '최선', '추진', '추천', '충분히', '충족', '해결', '호황', '확보', '활력', '활성화', '법정상속', '로슈그룹', '고취', '띠었음', '벳이글벳', '개월안', '감소', '강세', '레그단비맙', '리바운딩', '윤철진', '유나이티드헬스케어', '박기효', '특성', '유병삼', '셀루메드셀루메드', '솔리암페톨', '풍림파마텍', '사트칼리예프', '세보프란', '이더펠리체', '프럼파스트', '리레이팅', '드래콘', '화훼이', '출렁거릴', '피엠밸류로', '매직마이크', '레그단미밥', '킨젠바이오텍', '훌라발루', '일진일퇴', '랩스커버리', '프론텍', '자이렘', '페길레이션', '이점', '티플랙스', '중요', '휴톡스', '엄주용

In [128]:
pos = cell_pos_50
neg = cell_neg_50

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",0


In [129]:
pos = cell_pos_60
neg = cell_neg_60

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-2


In [130]:
pos = cell_pos_65
neg = cell_neg_65

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-6


In [131]:
pos = cell_pos_70
neg = cell_neg_70

sampro_210924['senti'] = 0
for index, wordList in enumerate(sampro_210924['Contents_input']):
    for word in wordList:
        if word in pos:
            sampro_210924['senti'][index] += 1
        if word in neg:
            sampro_210924['senti'][index] -= 1

sampro_210924

,st_n,st_cd,ch_nm,date,title,text,views,length,description,url,date_d,ud_x,ud_y,Contents_input,senti
0,LG화학,51910,삼프로TV_경제의신과함께,2021092400,[증시 셔터맨] 상승 속에서 하락을 걱정하게 된 국내 증시 f. 유안타증권 박진희 부장,자위헌터증권의박진희우정과오늘마감이왕정리를좀해드리죠토스쭉정이안녕하세요이번에뵙겠습니다연...,5970,1129,삼프로TV주식대학매매의기술업종기초교육올인원패키지강의신청httpsbitly39vJba...,https://www.youtube.com/watch?v=bNy68APRT-U,2021-09-24,-1,-1,"[자위, 헌터, 증권, 박진희, 우정, 마감, 정리, 토스, 쭉정이, 연희, 가지,...",-2
